1. ## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import random
from numpy import inf

2. ## Parameter Setting

In [2]:
#PA in ACO 
init_pheromone = 0
max_iter = 100
ant = 10
rho = 0.05 # Pheromone Evaporation Rate
alpha = 1 # Pheromone Exponential Parameter
beta = 1 # Desirabillity Exponential Parameter

"""
for all iterations iter do
    for all parallel instances m parallel do
        for all cocal ant a do
            local pheromone = global pheromone
            construct solution
            local pheromone update
            end for
        end for

"""


'\nfor all iterations iter do\n    for all parallel instances m parallel do\n        for all cocal ant a do\n            local pheromone = global pheromone\n            construct solution\n            local pheromone update\n            end for\n        end for\n\n'

3. ## Import Data Set (Supply Chain Logistics Problem)

In [3]:
dataOL = pd.read_excel(r'sclp.xlsx', sheet_name='OrderList')
dataFR = pd.read_excel(r'sclp.xlsx', sheet_name='FreightRates')
dataWhCost = pd.read_excel(r'sclp.xlsx', sheet_name='WhCosts')
dataWhCap = pd.read_excel(r'sclp.xlsx', sheet_name='WhCapacities')


4. ## Data Pre-Processing

In [4]:
#1 OrderList

dataOL=dataOL.drop(columns=['Order Date','TPT','Ship ahead day count','Ship Late Day count'])

#2 Freight Rate
dataFR = dataFR[dataFR['Carrier'].isin(['V444_0','V444_1','V44_3'])]
# print(p_origin_port_availability)
dataFR = dataFR[dataFR['orig_port_cd'].isin(['PORT04','PORT05','PORT09'])]
dataFR = dataFR.drop(columns=['tpt_day_cnt','Carrier type'])
print(dataFR.groupby(['orig_port_cd','Carrier']).size().reset_index(name='Availability'))
#3 Warehouse Cost
dataWhCost = dataWhCost[dataWhCost['WH'].isin(['PLANT03','PLANT04','PLANT08','PLANT09','PLANT12','PLANT13','PLANT16'])]

#4 Warehouse Capacity
dataWhCap = dataWhCap[dataWhCap['Plant ID'].isin(['PLANT03','PLANT04','PLANT08','PLANT09','PLANT12','PLANT13','PLANT16'])]


  orig_port_cd Carrier  Availability
0       PORT04  V444_0            20
1       PORT04  V444_1            40
2       PORT05  V444_1            48
3       PORT09  V444_0             2


5. Main Function

In [20]:

order_carrier_combination,selected_carrier = [],[]
dataFR_sub = dataFR
dataOL['Status'] = True
dataFR_sub['Status'] = True
local_ant = 0

record_local_ant =[[],[]]
carrier_utilization = 0
crf_utilization = 0
total_utilization = 0
max_util = 100

#if iteration 0
f_data_ol_status = dataOL['Status'] == True 
#PORT
while total_utilization < max_util:
    if local_ant == 0:
        p_origin_port_availability = dataOL.where(f_data_ol_status).groupby(['Origin Port','Status']).size().reset_index(name='Availability')
        p_origin_port_availability['Pheromone'] = init_pheromone #[PHEROMONE] Pheromone Initialisation for PORT
    selected_port = selection(p_origin_port_availability,'[PORT SELECTION]') # [SELECTION] Selection Process for PORT

    #PLANT CODE
    if local_ant == 0:
        p_plant_availability = dataOL.where(f_data_ol_status).groupby(['Origin Port','Plant Code','Status']).size().reset_index(name='Availability')
        p_plant_availability['Pheromone'] = init_pheromone #[PHEROMONE] Pheromone Initialisation for PLANT CODE
        f_origin_port = p_plant_availability["Origin Port"] == selected_port # [FILTER] Selected Origin Port Filter
        p_plant_availability = p_plant_availability.where(f_origin_port).groupby(['Plant Code','Status']).sum().reset_index()
    selected_plant_code = selection(p_plant_availability,'[PLANT SELECTION]') # [SELECTION] Selection Process for PLANT CODE

    #ORDER ID
    if local_ant == 0:
        f_plant_from_dataOL = dataOL["Plant Code"] == selected_plant_code # [FILTER] Selected Plant Filter DataOL
        f_port_from_dataOL = dataOL["Origin Port"] == selected_port # [FILTER] Selected Origin Port Filter DataOL
        path_order_availability = dataOL.where(f_plant_from_dataOL & f_port_from_dataOL & f_data_ol_status).groupby(['Order ID','Service Level','Carrier','Status']).size().reset_index(name='Availability')
        # path_order_availability['Availability'] = 1
        path_order_availability['Pheromone'] = init_pheromone # [PHEROMONE] Pheromone Initialisation for ORDER ID
    selected_path_order = selection(path_order_availability,'[ORDER ID SELECTION]')

    #CARRIER
    use_carrier = True #Does the logistics requires carriers?
    order_carrier_status = False #Has the carrier been used?
    carrier_index,carrier_index_index = 0,0

    if local_ant == 0:
        f_carrier_carrier = dataFR_sub['Carrier'] == selected_path_order['Carrier'][0] # [FILTER]
        f_carrier_order_origin = dataFR_sub['orig_port_cd'] == selected_port# [FILTER]

    if selected_path_order['Carrier'][0] != 'V44_3':
        if local_ant == 0:
            f_carrier_sel = dataFR_sub['Status'] == True
            carrier_availability = dataFR_sub.where(f_carrier_order_origin & f_carrier_sel).groupby([dataFR_sub.index,'Carrier','orig_port_cd','svc_cd','minm_wgh_qty','max_wgh_qty','Status']).size().reset_index(name='Availability')
            carrier_availability['Availability'] = 1
            carrier_availability['Pheromone'] = init_pheromone # Pheromone Initialisation for Carrier
        selected_carrier = selection(carrier_availability,'[CARRIER SELECTION]')

    else:
        use_carrier = False #No Carriers Needed
        selected_carrier = 'CRF'
    temp_occ = []
    order_carrier_status = False
    if use_carrier:
        if order_carrier_combination:
            for index,occ in enumerate(order_carrier_combination):
                if order_carrier_status == False:
                    if selected_path_order['Order ID'][0] == occ[0][0] and selected_path_order['Carrier'][0] == occ[0][1]:
                        # print('HERE3')
                        if order_carrier_status == False:
                            for index_,c_index in enumerate(occ[3]):
                                if c_index[0] == selected_carrier['level_0'][0]:
                                    order_carrier_status = True
                                    # print('HERE2')

                            if not order_carrier_status:
                                idx_lst = [selected_carrier['level_0'][0],init_pheromone,'Unoccupied']
                                temp_occ[index][3].append(idx_lst)
                                order_carrier_status = True
                                # print('HERE')
                                carrier_index_index = index_
                            
                            else:
                                if temp_occ[index][3][index_][2] == 'Occupied':
                                    print('[Carrier Is Occupied]')

                    else:
                        genesis_list = []
                        #print meow
                        idx_lst = [selected_carrier['level_0'][0],init_pheromone,'Unoccupied']
                        genesis_list.append(idx_lst)
                        order_carrier = [selected_path_order['Order ID'][0],selected_path_order['Carrier'][0],selected_carrier['Size'][0],genesis_list]
                        temp_occ.append(order_carrier)
                        carrier_index = index
                        order_carrier_status = True

        else:
            genesis_list = []
            idx_lst = [selected_carrier['level_0'][0],init_pheromone,'Unoccupied']
            genesis_list.append(idx_lst)
            order_carrier = [selected_path_order['Order ID'][0],selected_path_order['Carrier'][0],selected_carrier['Size'][0],genesis_list]
            temp_occ.append(order_carrier)

    #COST CALCULATION
    total_cost = 0
    if(use_carrier):
        t_cost = transportation_cost(use_carrier,selected_path_order,selected_carrier,dataOL,dataFR_sub)
    else:
        t_cost = 0.00

    if(type(t_cost) is not str):
        wh_cost,dataWhCap = warehouse_cost(dataWhCost,dataWhCap,selected_path_order,selected_plant_code,dataOL)
        # print('[TRANSPORTATION COST]\t$',"{:.2f}".format(t_cost))
        total_cost = t_cost+wh_cost
    else:
        # print('[TRANSPORTATION COST]\t$',t_cost)
        total_cost = 'INVALID ROUTE'
        temp_occ[carrier_index][3][carrier_index_index][2] = 'INVALID'

    # print('[WAREHOUSE COST]\t$',"{:.2f}".format(wh_cost))
        
    # if(type(total_cost) is str):
        # print('[TOTAL COST]\t\t$',total_cost,' due to ',t_cost)
    # else:
        # print('[TOTAL COST]\t\t$',"{:.2f}".format(total_cost))


    #AVAILABILITY UPDATE
    if(type(t_cost) is not str):
        if (type(selected_carrier) is str and crf_utilization < (max_util-carrier_utilization)):
            data_OL_index = dataOL[dataOL['Order ID']==selected_path_order['Order ID'][0]].index.values #filter index finding
            dataOL.at[data_OL_index,'Status'] = False
            record_local_ant[1].append(selected_path_order['Order ID'][0])
            crf_utilization+=1
            total_utilization+=1
        
        if use_carrier:
            if temp_occ[carrier_index][3][carrier_index_index][2] == 'Unoccupied' and carrier_utilization < 110:
                #Order ID Status
                data_OL_index = dataOL[dataOL['Order ID']==selected_path_order['Order ID'][0]].index.values #filter index finding
                dataOL.at[data_OL_index,'Status'] = False
                #Carrier Status
                dataFR_sub.at[selected_carrier['level_0'][0],'Status'] = False
                # if use_carrier:
                temp_occ[carrier_index][3][carrier_index_index][2] = 'Occupied' ##carrier tu nanti kena filter by occupied
                order_carrier_combination.append(temp_occ)
                record_local_ant[0].append(temp_occ)
                carrier_utilization+=1
                total_utilization+=1


        
    # print(order_carrier_combination)
        print(total_utilization,' [ ',selected_path_order['Order ID'][0],' ] ', total_cost)
        # print('=================================================')


    #PHEROMONE UPDATE
    #QUEEN ANT


1  [  1447386399.7  ]  277.1438551885316
2  [  1447390850.7  ]  149.11914671157965
3  [  1447373080.7  ]  225.34402496923323
4  [  1447373502.7  ]  156.80307316251492
5  [  1447272972.7  ]  3243.184354816769
6  [  1447248904.7  ]  569.4387462634209
7  [  1447423389.7  ]  1814.3463375986807
8  [  1447306544.7  ]  1875.4268552506735
9  [  1447223802.7  ]  8232.420091977847
10  [  1447151459.7  ]  408.6210899732442
11  [  1447184604.7  ]  354.4887957634413
12  [  1447423387.7  ]  47011.5601578595
13  [  1447288727.7  ]  1493.5104592310827
14  [  1447304799.7  ]  148.52304289650752
15  [  1447182884.7  ]  2746.900040747951
16  [  1447423385.7  ]  20230.955998599547
17  [  1447152942.7  ]  129.14216723148905
18  [  1447168927.7  ]  1449.9701946596676
19  [  1447353639.7  ]  3252.4993888660274
20  [  1447209353.7  ]  6381.315825633569
21  [  1447401849.7  ]  182.678167743788
22  [  1447373477.7  ]  150.59305046300938
23  [  1447320328.7  ]  17862.095291344438
24  [  1447182388.7  ]  4359.953

In [22]:
# print(temp_occ)
# print(record_local_ant)
# for index,occ in enumerate(order_carrier_combination):
#     print(occ[0][0])dataOL

# print(selected_port)
# print('===================')
# print(selected_plant_code)
# print('===================')
# print(selected_path_order)
# print(selected_path_order['Carrier'][0])
# print('===================')
# print(selected_carrier)
# print('===================')
# print(temp_occ)
# print(record_local_ant)
# print(use_carrier)
# print(type(t_cost))
print(order_carrier_combination)
print(record_local_ant[1])
print(record_local_ant[0])
# x = dataOL.where(dataOL['Status'] == False).groupby(['Plant Code']).size().reset_index(name='Availability')
# print(x)
# print(dataOL.where(dataOL['Status'] == True).groupby.(['Order ID']).reset_index())

[[[1447386399.7, 'V444_1', 60, [[861.0, 0, 'Occupied']]]], [[1447373080.7, 'V444_1', 59, [[185.0, 0, 'Occupied']]]], [[1447248904.7, 'V444_0', 58, [[864.0, 0, 'Occupied']]]], [[1447423389.7, 'V444_0', 57, [[193.0, 0, 'Occupied']]]], [[1447151459.7, 'V444_1', 56, [[869.0, 0, 'Occupied']]]], [[1447423387.7, 'V444_0', 55, [[190.0, 0, 'Occupied']]]], [[1447423385.7, 'V444_0', 54, [[1137.0, 0, 'Occupied']]]], [[1447152942.7, 'V444_1', 53, [[877.0, 0, 'Occupied']]]], [[1447168927.7, 'V444_0', 52, [[1134.0, 0, 'Occupied']]]], [[1447258890.7, 'V444_0', 51, [[1122.0, 0, 'Occupied']]]], [[1447162860.7, 'V444_0', 50, [[1125.0, 0, 'Occupied']]]], [[1447226237.7, 'V444_0', 49, [[878.0, 0, 'Occupied']]]], [[1447370727.7, 'V444_0', 48, [[1133.0, 0, 'Occupied']]]], [[1447275661.7, 'V444_0', 47, [[886.0, 0, 'Occupied']]]], [[1447242378.7, 'V444_0', 46, [[891.0, 0, 'Occupied']]]], [[1447274826.7, 'V444_0', 45, [[1123.0, 0, 'Occupied']]]], [[1447379010.7, 'V444_0', 44, [[876.0, 0, 'Occupied']]]], [[14471

In [24]:








# f_order = dataOL['Order ID'] == 1447305010.7
# print(f_order)
# target = dataOL.where(f_order).groupby([dataOL.index,'Order ID','Service Level','Carrier','Status']).sum()
# print(target)
dataWhCap
# dataOL.where(f_order).groupby([dataOL.index,'Order ID','Service Level','Carrier','Status']).sum()

0       False
1       False
2       False
3       False
4       False
        ...  
9210    False
9211    False
9212    False
9213    False
9214    False
Name: Order ID, Length: 9215, dtype: bool
                                               Product ID  Unit quantity  \
    Order ID     Service Level Carrier Status                              
393 1.447305e+09 CRF           V44_3   1.0      1696748.0          317.0   

                                                 Weight  
    Order ID     Service Level Carrier Status            
393 1.447305e+09 CRF           V44_3   1.0     0.358505  


,,,,,Product ID,Unit quantity,Weight
,Order ID,Service Level,Carrier,Status,,,
393,1.447305e+09,CRF,V44_3,1.0,1696748.0,317.0,0.358505


6. ## Functions

In [5]:
def selection(dat,type_sel):
    # print('============================================================================================================================')
    # print(dat)
    #parameter sequence: target, availability, pheromone
    
    #pheromone
    cumulative_pheromone,cum_p_segment = 0,0
    start_range,end_range = 0,0
    selection_list = []
    data_length = 0
    probability = 0
    target,target_index = '',0

    # Eliminate Routes With 0 Availability
    for index,row in dat.iterrows():
        if row['Status'] == False:
            dat = dat.drop(index)
        else:
            cumulative_pheromone += row['Pheromone']
            data_length += 1
            # print(data_length)
    
    #Normalize the Routes in a Uniform Roulette Wheel
    for index,row in dat.iterrows():

        cur_list=[]
        # Will be executed during the first iteration of ACO
        if dat['Pheromone'].sum() == 0:
            end_range = (index+1)/data_length
        # The next iterations
        else:
            # .index
            end_range = (cum_p_segment+row['Pheromone']/cumulative_pheromone)
            cum_p_segment += row['Pheromone']
            
        cur_list = [row[0],start_range,end_range]
        selection_list.append(cur_list)
        start_range = end_range

    roulette_value = random.uniform(0, 1)
    for sel_ in selection_list:
        if sel_[1] <= roulette_value and sel_[2] >= roulette_value:
            probability = "{:.2f}".format((sel_[2]-sel_[1])*100)
            target = sel_[0]

    # print(type_sel,'\tSelected Target is ',target,' with the Probability of ',probability,'% [ Roulette Value : ',roulette_value," ]")
    if dat.columns[0] == 'Order ID':
        f_order = dat['Order ID'] == target
        target = dat.where(f_order).groupby(['Order ID','Service Level','Carrier']).sum().reset_index()
        target['Size'] = data_length
        
    elif dat.columns[1] == 'Carrier':
        f_carrier = dat['level_0'] == target
        target = dat.where(f_carrier).groupby(['level_0','Carrier']).sum().reset_index()
        target['Size'] = data_length
        
    # print(target)

    return target

In [6]:
def objective_function():
    '''
    compare between the ants within a single iteration to find the queen ant
    min(sigma) = WC + TC
    '''
    fitness = 0
    return fitness

In [7]:
def warehouse_cost(dataWhCost,dataWhCap,selected_path_order,selected_plant_code,dataOL):

    data_Cap = dataWhCap.loc[dataWhCap['Plant ID'] == selected_plant_code].reset_index()
    data_Cap_index = dataWhCap.loc[dataWhCap['Plant ID'] == selected_plant_code]
    data_Cap_index = data_Cap_index.index[0]
    data_Cost = dataWhCost.loc[dataWhCost['WH'] == selected_plant_code].reset_index()
    data_OL = dataOL.loc[dataOL['Order ID'] == selected_path_order['Order ID'][0]].reset_index()
    wh_cost = 0

    if data_Cap['Daily Capacity '][0] > 0:
        counter = data_Cap['Daily Capacity '][0]-1
        wh_cost = data_OL['Unit quantity'][0]*data_Cost['Cost/unit'][0]
        dataWhCap.at[data_Cap_index,'Daily Capacity '] = counter

    else:
        wh_cost = 'Warehouse is Full!'

    return wh_cost,dataWhCap

In [8]:
def transportation_cost(use_carrier,selected_path_order,selected_carrier,dataOL,dataFR_sub):
    
    transportation_cost = 0
    data_OL = dataOL.loc[dataOL['Order ID'] == selected_path_order['Order ID'][0]].reset_index()
    data_FR = dataFR_sub.loc[selected_carrier['level_0'][0]]

    if use_carrier:

        if data_OL['Weight'][0] < data_FR['minm_wgh_qty']:
            transportation_cost = 'Not Enough Weight'
        elif data_OL['Weight'][0] > data_FR['max_wgh_qty']:
            transportation_cost = 'Exceeded Weight'
            
        else:
            if data_FR['mode_dsc'] == 'GROUND':
                transportation_cost = (data_FR['rate']/data_FR['max_wgh_qty'])*data_OL['Weight'][0]
            else:
                transportation_cost = data_FR['rate']*data_OL['Weight'][0]

            if transportation_cost < data_FR['minimum cost']:
                transportation_cost = data_FR['minimum cost']

    return transportation_cost